In [ ]:
import os, re
import numpy as np
import pandas as pd
import cudf, cupy
import gc

import matplotlib.pyplot as plt

In [ ]:
USE_FLOAT32 = True
FILTER_ZERO_WEIGHT = False

In [ ]:
%%time
data = cudf.read_csv("../input/jane-street-market-prediction/train.csv")
data.head()

In [ ]:
if USE_FLOAT32:
    float64_cols = data.select_dtypes('float64').columns.tolist()
    data[float64_cols] = data[float64_cols].astype('float32')

In [ ]:
if FILTER_ZERO_WEIGHT:
    data = data[data['weight'] > 0.0]

## Feature Engineering
All the features are computed
- without using the target
- without looking forward

In [ ]:
BASE_FEATURES = [f for f in data.columns if re.match('^feature_', f)]
FEATURES = [f for f in data.columns if re.match('^feature_', f)]

### Previous days FE

Not yet implemented for limits i memory

### Intra-day FE
Not yet implemented for limits i memory

### Row-wise FE
Calculate aggregations (mean, std, min, max) on variables with the same tag.

In [ ]:
features_tag = pd.read_csv("../input/jane-street-market-prediction/features.csv").set_index('feature').astype(int)

for it, t in enumerate(features_tag.columns):
    print("%s\t%d" % (t, features_tag[t].sum()))
    cols = features_tag[features_tag[t] > 0].index
    # mean
    name = 'feature_%s_mean' % t
    data[name] = data[cols].mean(axis=1, skipna=True)
    FEATURES += [name]
    gc.collect()
    
    # std
    name = 'feature_%s_std' % t
    data[name] = data[cols].std(axis=1, skipna=True)
    FEATURES += [name]
    gc.collect()
    
    # min
    name = 'feature_%s_min' % t
    data[name] = data[cols].min(axis=1, skipna=True)
    FEATURES += [name]
    gc.collect()
    
    # max
    name = 'feature_%s_max' % t
    data[name] = data[cols].max(axis=1, skipna=True)
    FEATURES += [name]
    gc.collect()

FEATURES = list(set(FEATURES))

### Save

In [ ]:
data.to_parquet('data.prq')